In [42]:
import boto3
from botocore.client import Config
import io
import json
import tqdm

In [1]:
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''
REGION_NAME = 'us-west-2'

# TEXTVQA

In [11]:
data = json.load(open('/media/abiten/4TB/Datasets/textvqa/train/TextVQA_0.5.1_train.json', 'r'))

In [51]:
splits = ['train', 'val', 'test']
bucket = 'stvqa'

for split in splits:
    ocr_response = {}
    data = json.load(open('/media/abiten/4TB/Datasets/textvqa/{}/TextVQA_0.5.1_{}.json'.format(split, split), 'r'))
    img_ids = list(set([d['image_id'] for d in data['data']]))
    config = Config(retries=dict(max_attempts=5))
    rek_client = boto3.client('rekognition', aws_access_key_id=AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                            region_name=REGION_NAME,
                           config=config)
    if split == 'test':
        aux_path = 'test_images'
    else:
        aux_path = 'train_images'
    
    for img_id in tqdm.tqdm(img_ids):

        if split == 'val':
            file_name = 'textvqa/train/{}/{}.jpg'.format(aux_path, img_id)
        else:
            file_name = 'textvqa/{}/{}/{}.jpg'.format(split, aux_path, img_id)

        response = rek_client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':file_name}})
        ocr_response[img_id] = response
        
    json.dump(ocr_response, open('/media/abiten/4TB/Datasets/textvqa/{}_amazon_ocr.json'.format(split), 'w'))

100%|██████████| 3289/3289 [1:19:09<00:00,  1.44s/it]


# STVQA

In [71]:
import shutil
import os

In [77]:
splits = ['train', 'test']
# splits = ['test']
bucket = 'stvqa'
config = Config(retries=dict(max_attempts=5))
rek_client = boto3.client('rekognition', aws_access_key_id=AWS_ACCESS_KEY_ID,
                        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                        region_name=REGION_NAME,
                       config=config)
for split in splits:
    ocr_response = {}
    data = json.load(open('/media/abiten/4TB/Datasets/st-vqa/{}_task_3.json'.format(split, split), 'r'))
    img_ids = list(set([d['file_path'] for d in data['data']]))
    config = Config(retries=dict(max_attempts=5))

    for img_id in tqdm.tqdm(img_ids):
        try:
            if split == 'test':
                file_name = 'st-vqa/{}/task3/{}'.format(split, img_id)
            else:
                file_name = 'st-vqa/{}/{}'.format(split, img_id)

            response = rek_client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':file_name}})
            ocr_response[img_id] = response
        except:
            print(img_id)
    json.dump(ocr_response, open('/media/abiten/4TB/Datasets/st-vqa/{}_amazon_ocr.json'.format(split), 'w'))

100%|██████████| 2971/2971 [56:49<00:00,  1.15s/it] 


In [75]:
len(ocr_response)

9354

# We need to replace COCO-Text with the original size images otherwise Amazon OCR doesnt work well

In [72]:
coco_path = '/media/abiten/4TB/Datasets/COCO_raw/train2014'
for split in splits:
    data = json.load(open('/media/abiten/4TB/Datasets/st-vqa/{}_task_3.json'.format(split, split), 'r'))
    coco_text = list(set([d['file_path'].split('/')[1] for d in data['data'] if 'coco-text' in d['file_path']]))
    for ct_img in coco_text:
        if split == 'test':
            file_name = '{}/task3/coco-text/{}'.format(split, ct_img)
        else:
            file_name = '{}/coco-text/{}'.format(split, ct_img)
        org_img = os.path.join(coco_path, ct_img)
        shutil.copy(org_img, os.path.join('/media/abiten/4TB/Datasets/st-vqa/', file_name))
#     orginal_img = '/media/abiten/4TB/Datasets/COCO_raw/train2014'